## Basic Background Subtraction using [OpenCV](https://www.opencv.org/)

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://www.opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use [CMake](https://www.cmake.org/).

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
// #include <vector>
#include <stdexcept>
// #include <sstream>
// #include <string>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Open the webcam

We are going to use the [`cv::VideoCapture` class](https://docs.opencv.org/master/d8/dfe/classcv_1_1VideoCapture.html) to grab frames from the webcam.

In [5]:
// Open the default camera (see the 0 below)
VideoCapture webcam(0);

// Check VideoCapture documentation.
if (!webcam.isOpened())
    throw runtime_error("OpenCV found no webcam, the program will terminate");

## Display the images from the camera

We create a window to display the images from the webcam

In [6]:
namedWindow("Webcam", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Background", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Subtraction", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Foreground mask", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Foreground", WINDOW_GUI_EXPANDED); // Create a window

We display the images in a loop. We also need the background. We pick up the backbround by pressing the 'b' key on the keyboard.

In [7]:
Mat background;

As we'll create a binary mask, I want to do some cleaning using mathematical morphology

In [8]:
Mat element = getStructuringElement(MORPH_CROSS,
                        Size(15, 15), 
                        Point(2, 2));

As we'll use a threshold, I'll add a trckbar to control it.

In [9]:
int threshold_value = 128;
createTrackbar( "Threshold: ", "Webcam", &threshold_value, 255, NULL );

In [10]:
int key = -1;
while (key != 27 && key != 'q')
{
    // Grab a new frame
    Mat frame;
    webcam >> frame;
    
    // Make sure everything went well
    if (frame.empty())
    {
        webcam.release(); // We are now done with the camera, stop it
        throw runtime_error("OpenCV cannot grab a new frame from the camera, the program will terminate");
    }
    
    // Display the image
    imshow("Webcam", frame);

    // This is the background
    if (key == 'b')
    {
        background = frame;
        cv::cvtColor(background, background, COLOR_BGR2GRAY);
        imshow("Background", background);
        background.convertTo(background, CV_32F);
    }
    // This is not the background
    else
    {
        // The background exists
        if (!background.empty())
        {
            // Convert in greyscale
            Mat grey_frame;
            cv::cvtColor(frame, grey_frame, COLOR_BGR2GRAY);
            grey_frame.convertTo(grey_frame, CV_32F);
            
            // Compute the foreground in colour
            Mat foreground;
            foreground = background - grey_frame;
            foreground = abs(foreground);
            foreground.convertTo(foreground, CV_8U);
            imshow("Subtraction", foreground);

            // Apply a threshold to generate the foreground mask
            Mat foreground_mask;
            threshold( foreground, foreground_mask, threshold_value, 255,  THRESH_BINARY);

            morphologyEx(foreground_mask, foreground_mask, MORPH_CLOSE, element);
            morphologyEx(foreground_mask, foreground_mask, MORPH_OPEN, element);

            imshow("Foreground mask", foreground_mask);
            //cv::normalize(foreground_mask, foreground_mask, 1, 0, NORM_MINMAX);

            // Apply the foreground mask
            Mat clean;
            frame.copyTo(clean, foreground_mask);
            imshow("Foreground", clean);
        }
    }    
    
    key = waitKey(1);
}

We don't need the window, destroy it

In [11]:
destroyAllWindows(); // Destroy all the created windows

We are now done with the camera, stop it

In [12]:
webcam.release();

In [ ]:
![Screenshot](../screenshot1.png)